<a href="https://colab.research.google.com/github/StarBlossom99/Pytorch_Study/blob/main/2022_0118/pytorch_study_autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5,3)
b = torch.randn(3)
w.requires_grad_(True)
b.requires_grad_(True)

z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

In [30]:
print(w)
print(b)

tensor([[-0.1736, -0.4479,  0.2425],
        [-0.5950,  0.4032,  1.1620],
        [-0.1828,  1.3298,  0.5295],
        [ 3.1621, -0.7437, -0.6221],
        [-1.2729,  1.4573, -0.6950]], requires_grad=True)
tensor([ 0.1935, -2.2621, -0.2318], requires_grad=True)


requires_grad의 값은 텐서를 생성할 때 설정하거나 나중에 x.requires_grad_(true) 메소드를 사용하여 나중에 설정할 수도 있다.

In [31]:
print('Gradient function for z = ', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z =  <AddBackward0 object at 0x7f0114bb4ad0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f0114bb4f90>


In [32]:
loss.backward()
print(w.grad)
print(b.grad)

print(w)
print(b)

tensor([[0.2520, 0.1448, 0.1984],
        [0.2520, 0.1448, 0.1984],
        [0.2520, 0.1448, 0.1984],
        [0.2520, 0.1448, 0.1984],
        [0.2520, 0.1448, 0.1984]])
tensor([0.2520, 0.1448, 0.1984])
tensor([[-0.1736, -0.4479,  0.2425],
        [-0.5950,  0.4032,  1.1620],
        [-0.1828,  1.3298,  0.5295],
        [ 3.1621, -0.7437, -0.6221],
        [-1.2729,  1.4573, -0.6950]], requires_grad=True)
tensor([ 0.1935, -2.2621, -0.2318], requires_grad=True)


requires_grad 속성의 값이 True로 설정된 노드들의 grad 속성만 구할 수 있고 다른 노드에는 변화도가 유효하지 않다. 성능상의 이유로 주어진 그래프에서의 backward를 사용한 변화도 계산은 한 번만 수행할 수 있고, 동일한 그래듶에서 여러번의 backward 호출이 필요하면, backward 호출시에 retrain_graph=True를 전달해야 한다.

In [34]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
  z = torch.matmul(x, w) + b
print(z.requires_grad)


z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)


True
False
False


requires_grad 속성 값이 True인 텐서의 변화도 추적을 멈추기 위해서는 torch.no_grad() 블록을 사용하거나 z.detach()를 이용하는 방법이 있다.

변화도 추척을 멈춰야 하는 이유는 불필요한 텐서의 연산을 막아 연산 속도를 향상시키려는 목적과 일부 매개변수를 고정되게 사용하고 싶은 경우가 있을 수 있기 때문이다.

autograd는 데이터에 실행된 모든 연산들의 기록을 Function 객체롤 구성된 방향성 비순환 그래프에 저장한다. 방향성 비순환 그래프(Directed Acyclic Graph)를 통해 뿌리(결과)에서부터 잎(입력)까지 추적하면 연쇄법칙에 따라 변화도를 자동으로 계산할 수 있다. 

forward propagation ->> 결과 텐서 계산, gradient function을 유지

backward propagation ->> 각 텐서의 .grad_fn 으로부터 변화도를 계산하고, .grad 속성에 계산결과를 쌓고 연쇄법칙을 사용하여 모든 텐서들에게 propagate



In [35]:
inp = torch.eye(5, requires_grad=True)
print(inp)

out = (inp+1).pow(2)
print(out)


tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)
tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)


torch.eye(size) size * size 행렬 생성인데 대각선만 1이고 나머지는 0

In [36]:
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
